In [79]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file bbmodcache already exists.
Error occurred while processing: bbmodcache.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18767  100 18767    0     0   259k      0 --:--:-- --:--:-- --:--:--  269k


In [80]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [81]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )


In [82]:
ROOM_CONTENTS = {
    'cemetery of ash'     : {'coiled sword'},
    'firelink shrine'     : {'estus flask'},
    'locked tower'        : {'uchiagatana'},
    'high wall of lothric': {'cell key', 'lothric banner'},
    'farron keep'         : {'soul of the blood of the wolf'},
}

ITEM_WEIGHT = {
                 'coiled sword' : 5,
                  'estus flask' : 3,
                  'uchiagatana' : 4,
                     'cell key' : 2,
'soul of the blood of the wolf' : 6,
               'lothric banner' : 4
}

DOORS = [
    Door( 'cemetery of ash', 'firelink shrine' ),
    Door( 'firelink shrine', 'high wall of lothric' ),
    Door( 'firelink shrine', 'locked tower', doorkey='cell key', locked=True ),
    Door( 'high wall of lothric', 'farron keep', doorkey='lothric banner', locked=True ),
]


In [83]:
from copy import deepcopy

class RobotWorker( SearchProblem ):

    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions( self, state ):

        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough
        for i in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHT[i]:
                actions.append( ("pick up", i))

        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
            elif door.locked==True and robot_location in door.goes_between:
                if door.doorkey in state.robot.carried_items:
                    actions.append( ("move to", door.other_loc[robot_location]) )

        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

In [84]:
rob = Robot('cemetery of ash', [], 10 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {"firelink shrine":{"soul of the blood of the wolf", "coiled sword"},
                        "farron keep":{"uchiagatana", "estus flask"}}

RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )

In [85]:
poss_acts = RW_PROBLEM_1.possible_actions( RW_PROBLEM_1.initial_state )
poss_acts

[('pick up', 'coiled sword'), ('move to', 'firelink shrine')]

In [86]:
for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM_1.successor( RW_PROBLEM_1.initial_state, act )
    RW_PROBLEM_1.display_state(next_state)
    print()

Action ('pick up', 'coiled sword') leads to the following state:
Robot location: cemetery of ash
Robot carrying: ['coiled sword']
Room contents: {'cemetery of ash': set(), 'firelink shrine': {'estus flask'}, 'locked tower': {'uchiagatana'}, 'high wall of lothric': {'lothric banner', 'cell key'}, 'farron keep': {'soul of the blood of the wolf'}}

Action ('move to', 'firelink shrine') leads to the following state:
Robot location: firelink shrine
Robot carrying: []
Room contents: {'cemetery of ash': {'coiled sword'}, 'firelink shrine': {'estus flask'}, 'locked tower': {'uchiagatana'}, 'high wall of lothric': {'lothric banner', 'cell key'}, 'farron keep': {'soul of the blood of the wolf'}}



In [87]:
def goal_items_heuristic(state):
    items_needed = 0
    for room, contents in goal_item_locations.items():
        for i in contents:
            if not i in state.room_contents[room]:
                items_needed += 1
    return items_needed

def minimize_visits_heuristic(state):
    visits_needed = 0
    for room, contents in goal_item_locations.items():
        items_needed = contents - state.room_contents[room]
        visits_needed += len(items_needed)
    return visits_needed

def keys_carried_heuristic(state):
    keys_needed = 0
    for door in state.doors:
        if door.locked and not door.doorkey in state.robot.carried_items:
            keys_needed += 1
    return keys_needed

In [88]:
import pandas as pd

# Initialize lists to store results
methods = ['BF/FIFO', 'DF/LIFO']
average_runtimes = []
average_path_lengths = []

# Run each search method 10 times
for method in methods:
    method_runtimes = []
    method_path_lengths = []
    for _ in range(1):
        result = search(RW_PROBLEM_1, method, 100000, loop_check=True, heuristic=keys_carried_heuristic, return_info=True)
        method_runtimes.append(result['search_stats']['time_taken'])
        method_path_lengths.append(result['result']['path_length'])
    average_runtimes.append(sum(method_runtimes) / len(method_runtimes))
    average_path_lengths.append(sum(method_path_lengths) / len(method_path_lengths))

# Create a DataFrame to store the results
df = pd.DataFrame({
    'Method': methods,
    'Average Runtime (s)': average_runtimes,
    'Average Path Length': average_path_lengths
})

# Display the DataFrame
print(df)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=keys_carried_heuristic
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
..................
:-)) *SUCCESS* ((-:

Path length = 26
Goal state is:
Robot location: firelink shrine
Robot carrying: ['lothric banner']
Room contents: {'cemetery of ash': set(), 'firelink shrine': {'soul of the blood of the wolf', 'coiled sword'}, 'locked tower': set(), 'high wall of lothric': set(), 'farron keep': {'estus flask', 'cell key', 'uchiagatana'}}
The action path to the solution is:
    ('move to', 'firelink shrine')
    ('move to', 'high wall of lothric')
    ('pick up', 'lothric banner')
    ('pick up', 'cell key')
    ('move to', 'firelink shrine')
    ('pick up', 'estus flask')
    ('move to', 'high wall of lothric')
    ('move to', 'farron keep')
